In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

stop_words=stopwords.words('english')
punc = string.punctuation
punc = list(punc)
punc.extend(['``',"''"])

remove_list=punc+stop_words

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.shape

(5000, 3)

In [5]:
df.head()

,id,comment_text,toxic
0,e617e2489abe9bca,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0
1,9250cf637294e09d,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0
2,ce1aa4592d5240ca,"Marya Dzmitruk was born in Minsk, Belarus in M...",0
3,48105766ff7f075b,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0
4,0543d4f82e5470b6,New Categories \r\n\r\nI honestly think that w...,0


In [6]:
text = df["comment_text"].tolist()
text

['"\r\n\r\n A barnstar for you! \r\n\r\n  The Defender of the Wiki Barnstar I like your edit on the Kayastha page. Lets form a solidarity group against those who malign the article and its subject matter. I propose the folloing name for the group.\r\n\r\nUnited intellectuals\' front of Kayastha ethinicty against racist or castist abuse (UIFKEARCA)   "',
 '"\r\n\r\nThis seems unbalanced.  whatever I have said about Mathsci, he has said far more extreme and unpleasant things about me (not to mention others), and with much greater frequency.  I\'m more than happy to reign myself in, if that\'s what you\'d like (ruth be told, I was just trying to get Mathsci to pay attention and stop being uncivil).  I would expect you to issue the same request to Mathsci.  \r\n\r\n If this is intentionally unbalanced (for whatever reason), please let me know, and I will voluntarily close this account and move on to other things.  I like wikipedia, and I have a lot to contribute in my own way, but there is

In [7]:
len(text)

5000

In [8]:
def preprocess(clean):
    clean = re.sub('\d*\.\d*\.\d*\.\d*', '', clean)
    clean = re.sub(r'(https?://\S+)', '', clean)
    clean = re.sub(r'\n', '', clean)
    clean = re.sub(r'\r','',clean)
    word_tokens = word_tokenize(clean.lower()) 
    words_list=[]
    for word in word_tokens:
        if word not in remove_list:
            words_list.append(word)
    
    #filtered_text = ' '.join(words_list)
    
    
    return words_list #filtered_text

In [9]:
new=[]

In [10]:
for i in text:
    x = preprocess(i)
    new.append(x)
new

[['barnstar',
  'defender',
  'wiki',
  'barnstar',
  'like',
  'edit',
  'kayastha',
  'page',
  'lets',
  'form',
  'solidarity',
  'group',
  'malign',
  'article',
  'subject',
  'matter',
  'propose',
  'folloing',
  'name',
  'group.united',
  'intellectuals',
  'front',
  'kayastha',
  'ethinicty',
  'racist',
  'castist',
  'abuse',
  'uifkearca'],
 ['seems',
  'unbalanced',
  'whatever',
  'said',
  'mathsci',
  'said',
  'far',
  'extreme',
  'unpleasant',
  'things',
  'mention',
  'others',
  'much',
  'greater',
  'frequency',
  "'m",
  'happy',
  'reign',
  "'s",
  "'d",
  'like',
  'ruth',
  'told',
  'trying',
  'get',
  'mathsci',
  'pay',
  'attention',
  'stop',
  'uncivil',
  'would',
  'expect',
  'issue',
  'request',
  'mathsci',
  'intentionally',
  'unbalanced',
  'whatever',
  'reason',
  'please',
  'let',
  'know',
  'voluntarily',
  'close',
  'account',
  'move',
  'things',
  'like',
  'wikipedia',
  'lot',
  'contribute',
  'way',
  'point',
  'contribut

In [11]:
len(new)

5000

In [12]:
count1 = 0
for listElem in new:
    count1 += len(listElem)

count1

178019

In [13]:
#4. Using a Counter, Find the Top Terms in the Data. Remove Contextual Stop Words.

In [14]:
words = list(itertools.chain.from_iterable(new))
words

['barnstar',
 'defender',
 'wiki',
 'barnstar',
 'like',
 'edit',
 'kayastha',
 'page',
 'lets',
 'form',
 'solidarity',
 'group',
 'malign',
 'article',
 'subject',
 'matter',
 'propose',
 'folloing',
 'name',
 'group.united',
 'intellectuals',
 'front',
 'kayastha',
 'ethinicty',
 'racist',
 'castist',
 'abuse',
 'uifkearca',
 'seems',
 'unbalanced',
 'whatever',
 'said',
 'mathsci',
 'said',
 'far',
 'extreme',
 'unpleasant',
 'things',
 'mention',
 'others',
 'much',
 'greater',
 'frequency',
 "'m",
 'happy',
 'reign',
 "'s",
 "'d",
 'like',
 'ruth',
 'told',
 'trying',
 'get',
 'mathsci',
 'pay',
 'attention',
 'stop',
 'uncivil',
 'would',
 'expect',
 'issue',
 'request',
 'mathsci',
 'intentionally',
 'unbalanced',
 'whatever',
 'reason',
 'please',
 'let',
 'know',
 'voluntarily',
 'close',
 'account',
 'move',
 'things',
 'like',
 'wikipedia',
 'lot',
 'contribute',
 'way',
 'point',
 'contributing',
 'project',
 'editors',
 'administrative',
 'leave',
 'aggressively',
 'rude'

In [15]:
len(words)

178019

In [16]:
freqTable={}

for word in words:
    if word in freqTable: 
        freqTable[word] += 1
            
    else: 
        freqTable[word] = 1 

In [17]:
{k: v for k, v in sorted(freqTable.items(),reverse=True, key=lambda item: item[1])}

{"'s": 1988,
 "n't": 1795,
 'article': 1673,
 'page': 1450,
 'wikipedia': 1388,
 'talk': 1167,
 'would': 1004,
 'please': 1003,
 'ass': 985,
 'fuck': 902,
 'one': 853,
 'like': 832,
 'also': 640,
 'think': 628,
 'see': 626,
 'know': 594,
 'edit': 557,
 'people': 548,
 'use': 539,
 "'m": 538,
 'name': 532,
 'may': 530,
 'articles': 481,
 'time': 470,
 'user': 413,
 "'re": 407,
 'even': 402,
 'could': 392,
 'make': 389,
 'information': 384,
 'deletion': 377,
 'suck': 375,
 "'ve": 373,
 'thanks': 372,
 'get': 367,
 'good': 366,
 'want': 364,
 'mexicans': 362,
 'well': 354,
 'image': 347,
 'wp': 340,
 'editing': 335,
 'way': 333,
 'edits': 331,
 'help': 329,
 'new': 325,
 'first': 317,
 'must': 317,
 'pages': 316,
 'sources': 314,
 'need': 309,
 'thank': 308,
 'say': 307,
 'many': 301,
 'really': 299,
 'deleted': 296,
 'source': 292,
 'used': 283,
 'fucking': 281,
 'work': 276,
 'section': 275,
 'go': 271,
 'find': 271,
 'take': 263,
 'made': 259,
 'look': 259,
 "'ll": 258,
 'discussion': 

In [18]:
#Top Meaningless symbols and Contextual Stop Words to Remove:
# "'s", "n't", "article", "page", "wikipedia", "edit", "'m", "articles", "'re", "'ve", "editing", "edits", "pages", "'ll"

In [19]:
#Update Frequency Table:

In [20]:
remove_list2 = (["'s", "n't", "article", "page", "wikipedia", "edit", "'m", "articles", "'re", "'ve", "editing", "edits", "pages", "'ll"])

In [21]:
words2=[]
for word in words:
    if word not in remove_list2:
        words2.append(word)

In [22]:
len(words2)

166129

In [23]:
freqTable2={}

for word in words2:
    if word in freqTable2: 
        freqTable2[word] += 1
            
    else: 
        freqTable2[word] = 1

In [24]:
{k: v for k, v in sorted(freqTable2.items(),reverse=True, key=lambda item: item[1])}

{'talk': 1167,
 'would': 1004,
 'please': 1003,
 'ass': 985,
 'fuck': 902,
 'one': 853,
 'like': 832,
 'also': 640,
 'think': 628,
 'see': 626,
 'know': 594,
 'people': 548,
 'use': 539,
 'name': 532,
 'may': 530,
 'time': 470,
 'user': 413,
 'even': 402,
 'could': 392,
 'make': 389,
 'information': 384,
 'deletion': 377,
 'suck': 375,
 'thanks': 372,
 'get': 367,
 'good': 366,
 'want': 364,
 'mexicans': 362,
 'well': 354,
 'image': 347,
 'wp': 340,
 'way': 333,
 'help': 329,
 'new': 325,
 'first': 317,
 'must': 317,
 'sources': 314,
 'need': 309,
 'thank': 308,
 'say': 307,
 'many': 301,
 'really': 299,
 'deleted': 296,
 'source': 292,
 'used': 283,
 'fucking': 281,
 'work': 276,
 'section': 275,
 'go': 271,
 'find': 271,
 'take': 263,
 'made': 259,
 'look': 259,
 'discussion': 258,
 'read': 256,
 'right': 255,
 'point': 254,
 'since': 248,
 'still': 248,
 'add': 245,
 'much': 244,
 'gay': 243,
 'content': 241,
 'someone': 241,
 'two': 240,
 'ytmnd': 238,
 'fact': 233,
 'link': 232,
 

In [25]:
#Next, remove all determined words/symbols from the actual list of reviews:

In [26]:
def postprocess(cleaner):
    words_list2=[]
    for word in cleaner:
        if word not in remove_list2:
            words_list2.append(word)    
    
    return words_list2

In [27]:
newer=[]

In [28]:
for i in new:
    x = postprocess(i)
    newer.append(x)
newer

[['barnstar',
  'defender',
  'wiki',
  'barnstar',
  'like',
  'kayastha',
  'lets',
  'form',
  'solidarity',
  'group',
  'malign',
  'subject',
  'matter',
  'propose',
  'folloing',
  'name',
  'group.united',
  'intellectuals',
  'front',
  'kayastha',
  'ethinicty',
  'racist',
  'castist',
  'abuse',
  'uifkearca'],
 ['seems',
  'unbalanced',
  'whatever',
  'said',
  'mathsci',
  'said',
  'far',
  'extreme',
  'unpleasant',
  'things',
  'mention',
  'others',
  'much',
  'greater',
  'frequency',
  'happy',
  'reign',
  "'d",
  'like',
  'ruth',
  'told',
  'trying',
  'get',
  'mathsci',
  'pay',
  'attention',
  'stop',
  'uncivil',
  'would',
  'expect',
  'issue',
  'request',
  'mathsci',
  'intentionally',
  'unbalanced',
  'whatever',
  'reason',
  'please',
  'let',
  'know',
  'voluntarily',
  'close',
  'account',
  'move',
  'things',
  'like',
  'lot',
  'contribute',
  'way',
  'point',
  'contributing',
  'project',
  'editors',
  'administrative',
  'leave',
 

In [29]:
len(newer)

5000

In [30]:
count2 = 0
for listElem in newer:
    count2 += len(listElem)

count2

166129

In [31]:
#5. Separate into Train and Test Sets

In [32]:
def condense(hello):
    filtered_text = ' '.join(hello)
    
    return filtered_text

In [33]:
comment_text2 = []

In [34]:
for i in newer:
    x = condense(i)
    comment_text2.append(x)
comment_text2

['barnstar defender wiki barnstar like kayastha lets form solidarity group malign subject matter propose folloing name group.united intellectuals front kayastha ethinicty racist castist abuse uifkearca',
 "seems unbalanced whatever said mathsci said far extreme unpleasant things mention others much greater frequency happy reign 'd like ruth told trying get mathsci pay attention stop uncivil would expect issue request mathsci intentionally unbalanced whatever reason please let know voluntarily close account move things like lot contribute way point contributing project editors administrative leave aggressively rude good editor really deserve people riding ass every time try certain things happily leave hands drama-prone think best ludwigs2",
 'marya dzmitruk born minsk belarus march 19 1992. mother olga nikolaevna moroz born baranovichi belarus father born brest belarus second child family parents divorced 1998 soon father remarried two children marya age 4 began gymnastics quit two yea

In [35]:
len(comment_text2)

5000

In [36]:
df = df.assign(comment_text2 = comment_text2)

In [37]:
df = df[['id', 'comment_text', 'comment_text2', 'toxic']]

In [38]:
df

,id,comment_text,comment_text2,toxic
0,e617e2489abe9bca,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",barnstar defender wiki barnstar like kayastha ...,0
1,9250cf637294e09d,"""\r\n\r\nThis seems unbalanced. whatever I ha...",seems unbalanced whatever said mathsci said fa...,0
2,ce1aa4592d5240ca,"Marya Dzmitruk was born in Minsk, Belarus in M...",marya dzmitruk born minsk belarus march 19 199...,0
3,48105766ff7f075b,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,talkback dear celestia,0
4,0543d4f82e5470b6,New Categories \r\n\r\nI honestly think that w...,new categories honestly think need add categor...,0
...,...,...,...,...
4995,60229df7b48ba6ff,"""\r\n\r\n Dildo, if you read my response corre...",dildo read response correctly never said going...,0
4996,36a645227572ec5c,"CALM DOWN, CALM DOWN, DON'T GET A BIG DICK",calm calm get big dick,1
4997,6d47fa39945ed6f5,In my opinion Dougweller is using his privileg...,opinion dougweller using privileges poorly per...,0
4998,de2e4c0d38db6e30,The style section has been expanded too. I did...,style section expanded remember placed tag,0


In [39]:
#5. Separate into Train (70%) and Test (30%) sets.

In [40]:
X_train, X_test, y_train, y_test = train_test_split(df['comment_text2'], df['toxic'], test_size=0.3, random_state=101)

In [42]:
#6. Use TF-IDF values for the terms as feature to get into a vector space model.

In [43]:
tfidf = TfidfVectorizer(max_features = 4000) #analyzer = 'word', ngram_range = (1,2),

In [44]:
X_train_fit = tfidf.fit_transform(X_train)
X_train_fit

<3500x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 72927 stored elements in Compressed Sparse Row format>

In [45]:
X_train_fit_df = pd.DataFrame(X_train_fit.todense(),columns=tfidf.get_feature_names())
X_train_fit_df.head()

,00,000,0000z,01,02,03,04,0422,05,06,...,zach,zero,zheng,мај,јул,јули,јун,јуни,கள,தம
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
X_test_fit = tfidf.transform(X_test)
X_test_fit

<1500x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 31453 stored elements in Compressed Sparse Row format>

In [47]:
X_test_fit_df = pd.DataFrame(X_test_fit.todense(),columns=tfidf.get_feature_names())
X_test_fit_df

,00,000,0000z,01,02,03,04,0422,05,06,...,zach,zero,zheng,мај,јул,јули,јун,јуни,கள,தம
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.433677,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
#7. Model Building: Support Vector Machine (SVC)

In [49]:
clf = svm.SVC(C=1, kernel='linear')

In [50]:
clf.fit(X_train_fit, y_train)

SVC(C=1, kernel='linear')

In [51]:
predict_train = clf.predict(X_train_fit)

In [52]:
predict_test = clf.predict(X_test_fit)

In [53]:
print(clf, '\n', classification_report(y_train, predict_train))
print(clf, '\n', classification_report(y_test, predict_test))

SVC(C=1, kernel='linear') 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3188
           1       1.00      0.63      0.77       312

    accuracy                           0.97      3500
   macro avg       0.98      0.82      0.88      3500
weighted avg       0.97      0.97      0.96      3500

SVC(C=1, kernel='linear') 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      1375
           1       0.94      0.41      0.57       125

    accuracy                           0.95      1500
   macro avg       0.95      0.70      0.77      1500
weighted avg       0.95      0.95      0.94      1500



In [54]:
print(len(predict_train), ",", len(predict_test))

3500 , 1500


In [55]:
#8. Model Evaluation: Accuracy, Recall, and f1_score of the Train set

In [56]:
train_accuracy = clf.score(X_train_fit, y_train)

train_recall = recall_score(y_train, predict_train)

train_precision = precision_score(y_train, predict_train)
train_f1score = (2*train_precision*train_recall)/(train_precision+train_recall)


print("Train Accuracy: ", train_accuracy, "\nTrain Recall: ", train_recall, "\nTrain f1_score", train_f1score)

Train Accuracy:  0.9671428571428572 
Train Recall:  0.6314102564102564 
Train f1_score 0.7740667976424361


In [57]:
#Recall value and f1_score aren't very good...

In [58]:
test_accuracy = clf.score(X_test_fit_df, y_test)

test_recall = recall_score(y_test, predict_test)

test_precision = precision_score(y_test, predict_test)
test_f1score = (2*test_precision*test_recall)/(test_precision+test_recall)


print("Test Accuracy: ", test_accuracy, "\nTest Recall: ", test_recall, "\nTest f1_score", test_f1score)

Test Accuracy:  0.9486666666666667 
Test Recall:  0.408 
Test f1_score 0.5698324022346368


In [60]:
#9. Adjust SVC parameter(s)

In [61]:
# Increasing the C value in the svm.SVC() algorithm increases the value for train_accuracy,
# and GREATLY increases the values of test_recall and test_f1score.
# After a few guess and checks(C=1,10,100,1000,10000) it looks like C=100 is a good value to move forward with. 

In [62]:
clf2 = svm.SVC(C=100, kernel='linear')

In [63]:
#10. Train again with the adjustment, and evaluate on Test Set

In [64]:
clf2.fit(X_train_fit, y_train)

SVC(C=100, kernel='linear')

In [70]:
predict2_train = clf2.predict(X_train_fit)
predict2_test = clf2.predict(X_test_fit)

In [71]:
print(clf, '\n', classification_report(y_train, predict2_train))
print(clf, '\n', classification_report(y_test, predict2_test))

SVC(C=1, kernel='linear') 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3188
           1       1.00      0.98      0.99       312

    accuracy                           1.00      3500
   macro avg       1.00      0.99      1.00      3500
weighted avg       1.00      1.00      1.00      3500

SVC(C=1, kernel='linear') 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      1375
           1       0.71      0.58      0.64       125

    accuracy                           0.95      1500
   macro avg       0.84      0.78      0.81      1500
weighted avg       0.94      0.95      0.94      1500



In [72]:
train_accuracy2 = clf2.score(X_train_fit, y_train)

train_recall2 = recall_score(y_train, predict2_train)

train_precision2 = precision_score(y_train, predict2_train)
train_f1score2 = (2*train_precision2*train_recall2)/(train_precision2+train_recall2)


print("Train Accuracy2: ", train_accuracy2, "\nTrain Recall2: ", train_recall2, "\nTrain f1_score2", train_f1score2)

Train Accuracy2:  0.9985714285714286 
Train Recall2:  0.9839743589743589 
Train f1_score2 0.9919224555735056


In [73]:
test_accuracy2 = clf2.score(X_test_fit, y_test)

test_recall2 = recall_score(y_test, predict2_test)

test_precision2 = precision_score(y_test, predict2_test)
test_f1score2 = (2*test_precision2*test_recall2)/(test_precision2+test_recall2)


print("Test Accuracy2: ", test_accuracy2, "\nTest Recall2: ", test_recall2, "\nTest f1_score2", test_f1score2)

Test Accuracy2:  0.9453333333333334 
Test Recall2:  0.584 
Test f1_score2 0.6403508771929824


In [74]:
# Recall and f1_score are poor, even with the C value update. 
# I did verify these values w/ C=1,10,100,1000,10000 and the results do not improve much with differing C values

In [75]:
#11. Hyperparameter Tuning

In [76]:
EST = svm.SVC(kernel='linear', class_weight='balanced')
gridvalues = {'C':[1,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]}

In [77]:
clf3 = GridSearchCV(estimator = EST, param_grid = gridvalues, scoring = 'recall')
clf3

GridSearchCV(estimator=SVC(class_weight='balanced', kernel='linear'),
             param_grid={'C': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110,
                               120, 130, 140, 150]},
             scoring='recall')

In [78]:
#12. Find the parameters with the best recall in Cross Validation

In [79]:
skf = StratifiedKFold(n_splits=5)
skf

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [80]:
y_all = y_train.append(y_test)
y_all

2654    0
2468    0
290     0
1463    1
4508    0
       ..
3412    1
4020    0
4635    0
1700    0
790     1
Name: toxic, Length: 5000, dtype: int64

In [81]:
y_all_zero = y_all.reset_index(drop=True)
y_all_zero

0       0
1       0
2       0
3       1
4       0
       ..
4995    1
4996    0
4997    0
4998    0
4999    1
Name: toxic, Length: 5000, dtype: int64

In [82]:
df5 = X_train_fit_df
df5

,00,000,0000z,01,02,03,04,0422,05,06,...,zach,zero,zheng,мај,јул,јули,јун,јуни,கள,தம
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
df5 = df5.append(X_test_fit_df)
df5

,00,000,0000z,01,02,03,04,0422,05,06,...,zach,zero,zheng,мај,јул,јули,јун,јуни,கள,தம
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
df5 = df5.reset_index(drop=True)
df5

,00,000,0000z,01,02,03,04,0422,05,06,...,zach,zero,zheng,мај,јул,јули,јун,јуни,கள,தம
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
df5 = df5.assign(toxic = y_all_zero)
df5

,00,000,0000z,01,02,03,04,0422,05,06,...,zero,zheng,мај,јул,јули,јун,јуни,கள,தம,toxic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [86]:
target = df5.loc[:,'toxic']

In [240]:
def train_model(train5, test5, fold_no):
    #X5 = [:4000]
    y5 = ['toxic']
    X_train5 = train5.iloc[:,:4000] #<- train5 is full data. figure out how to pull all column names except for "toxic" in X5
    y_train5 = train5[y5]
    X_test5 = test5.iloc[:,:4000]
    y_test5 = test5[y5]
    clf3.fit(X_train5,y_train5)
    predictions = clf3.predict(X_test5)
    print('Fold',str(fold_no),'Recall:',recall_score(y_test5,predictions),'f1_score',f1_score(y_test5,predictions),'Best Estimator',clf3.best_estimator_)

In [241]:
fold_no = 1

In [242]:
for train_index, test_index in skf.split(df5, target):
    train5 = df5.loc[train_index,:]
    test5 = df5.loc[test_index,:]
    train_model(train5,test5,fold_no) #<- pulling to definition above
    fold_no += 1

C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


Fold 1 Recall: 0.6206896551724138 f1_score 0.4615384615384615 Best Estimator SVC(C=60, class_weight='balanced', kernel='linear')


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


...

Fold 4 Recall: 0.6590909090909091 f1_score 0.44961240310077516 Best Estimator SVC(C=50, class_weight='balanced', kernel='linear')


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


Fold 5 Recall: 0.6477272727272727 f1_score 0.4634146341463415 Best Estimator SVC(C=70, class_weight='balanced', kernel='linear')


In [87]:
#13. Best Value of Parameter C: 50

In [88]:
# The best value of Parameter C is 50, which, when paired w/ StratifiedKfold tuning, yields a top Recall value of 67.8% and an f1_score of 46.3%.
# See results above in for loop.

In [89]:
#14. Predict and Evaluate Using the Best Estimator.

In [90]:
# As shown above in the for loop, The best estimator was C = 50.

In [91]:
# Now we can run a new program w/ C=50 on all values, and use StratifiedKFold w/ 5 folds once again.

In [92]:
clf4 = svm.SVC(C=50, kernel='linear', class_weight='balanced')

In [94]:
target = df5.loc[:,'toxic']
X_test5_all = []
predict5_all = []

In [95]:
def train_model(train5, test5, fold_no, X_test_all, predict_all):
    #X5 = [:4000]
    y5 = ['toxic']
    X_train5 = train5.iloc[:,:4000]
    y_train5 = train5[y5]
    X_test5 = test5.iloc[:,:4000]
    y_test5 = test5[y5]
    clf4.fit(X_train5,y_train5)
    predictions = clf4.predict(X_test5)
    X_test5_all = X_test_all.append(X_test5)
    predict5_all = predict_all.append(predictions)
    
    
    print('Fold',str(fold_no),'Recall:',recall_score(y_test5,predictions),'f1_score',f1_score(y_test5,predictions))
    return X_test5_all,#predict5_all

In [96]:
fold_no = 1

In [97]:
for train_index, test_index in skf.split(df5, target):
    train5 = df5.loc[train_index,:]
    test5 = df5.loc[test_index,:]
    X_test_all = X_test5_all
    predict_all = predict5_all
    train_model(train5, test5, fold_no, X_test_all, predict_all)
    fold_no += 1

C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fold 1 Recall: 0.6091954022988506 f1_score 0.4549356223175966


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fold 2 Recall: 0.632183908045977 f1_score 0.4867256637168141


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fold 3 Recall: 0.6781609195402298 f1_score 0.4627450980392156


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fold 4 Recall: 0.6590909090909091 f1_score 0.44961240310077516


C:\Users\Tim\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fold 5 Recall: 0.625 f1_score 0.5092592592592593


In [98]:
#15. Most Prominent Terms in the Toxic Comments

In [99]:
X_test5_all

[       00  000  0000z   01   02   03   04  0422   05   06  ...  zach  zero  \
 0     0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 1     0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 2     0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 3     0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 4     0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 ...   ...  ...    ...  ...  ...  ...  ...   ...  ...  ...  ...   ...   ...   
 995   0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 997   0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 998   0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 999   0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 1001  0.0  0.0    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...   0.0   0.0   
 
       zheng  мај  јул  јули  јун  јуни   கள   தம 

In [101]:
X_test5_all_2 = pd.concat(X_test5_all,join = 'inner')
X_test5_all_2

,00,000,0000z,01,02,03,04,0422,05,06,...,zach,zero,zheng,мај,јул,јули,јун,јуни,கள,தம
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
a = X_test5_all_2.index
print(*a, sep = "\n")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429


4566
4567
4568
4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4654
4655
4656
4657
4658
4659
4660
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4740
4741
4742
4743
4744
4745
4746
4747
4748
4749
4750
4751
4752
4753
4754
4755
4756
4757
4758
4759
4760
4761
4762
4763
4764
4765


In [103]:
predict5_all

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [104]:
predict5_all_2 = []

In [105]:
for i in predict5_all:
    listed = i.tolist()
    predict5_all_2 += listed

In [106]:
len(predict5_all_2)

5000

In [107]:
predict5_all_2

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [108]:
X_test5_all_2['toxic_pred'] = predict5_all_2
X_test5_all_2

,00,000,0000z,01,02,03,04,0422,05,06,...,zero,zheng,мај,јул,јули,јун,јуни,கள,தம,toxic_pred
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [109]:
HALO = X_test5_all_2.sort_index()
HALO

,00,000,0000z,01,02,03,04,0422,05,06,...,zero,zheng,мај,јул,јули,јун,јуни,கள,தம,toxic_pred
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [111]:
y_all

2654    0
2468    0
290     0
1463    1
4508    0
       ..
3412    1
4020    0
4635    0
1700    0
790     1
Name: toxic, Length: 5000, dtype: int64

In [112]:
HALO2 = HALO.set_index(y_all.index)
HALO2

,00,000,0000z,01,02,03,04,0422,05,06,...,zero,zheng,мај,јул,јули,јун,јуни,கள,தம,toxic_pred
2654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [113]:
HALO2 = HALO2.sort_index()
HALO2

,00,000,0000z,01,02,03,04,0422,05,06,...,zero,zheng,мај,јул,јули,јун,јуни,கள,தம,toxic_pred
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [114]:
df3 = df

In [115]:
df3['toxic_pred'] = HALO2['toxic_pred']
df3

,id,comment_text,comment_text2,toxic,toxic_pred
0,e617e2489abe9bca,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",barnstar defender wiki barnstar like kayastha ...,0,0
1,9250cf637294e09d,"""\r\n\r\nThis seems unbalanced. whatever I ha...",seems unbalanced whatever said mathsci said fa...,0,0
2,ce1aa4592d5240ca,"Marya Dzmitruk was born in Minsk, Belarus in M...",marya dzmitruk born minsk belarus march 19 199...,0,0
3,48105766ff7f075b,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,talkback dear celestia,0,1
4,0543d4f82e5470b6,New Categories \r\n\r\nI honestly think that w...,new categories honestly think need add categor...,0,0
...,...,...,...,...,...
4995,60229df7b48ba6ff,"""\r\n\r\n Dildo, if you read my response corre...",dildo read response correctly never said going...,0,0
4996,36a645227572ec5c,"CALM DOWN, CALM DOWN, DON'T GET A BIG DICK",calm calm get big dick,1,1
4997,6d47fa39945ed6f5,In my opinion Dougweller is using his privileg...,opinion dougweller using privileges poorly per...,0,0
4998,de2e4c0d38db6e30,The style section has been expanded too. I did...,style section expanded remember placed tag,0,0


In [116]:
df_filtered = df[df['toxic_pred'] == 1]

toxic_pred_comments = df_filtered["comment_text2"].tolist()

def process (token):
    words_list3=[]
    word_tokens = word_tokenize(token)
    for word in word_tokens:
        words_list3.append(word)
    return words_list3

toxic = []

for i in toxic_pred_comments:
    x = process(i)
    toxic.append(x)

words_toxic = list(itertools.chain.from_iterable(toxic))
words_toxic

['talkback',
 'dear',
 'celestia',
 'loser',
 'ca',
 'block',
 'forever',
 'admin',
 'ego',
 'hippie',
 'freak',
 'seen',
 'source',
 'gwern',
 'used',
 'ran',
 'across',
 'sites',
 'like',
 'would',
 'inappropriate',
 'use',
 'source',
 'theres',
 'fucking',
 'wiki',
 'insane',
 'person',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'ass',
 'fuck',
 'a

In [117]:
freqTable3={}

for word in words_toxic:
    if word in freqTable3: 
        freqTable3[word] += 1
            
    else: 
        freqTable3[word] = 1

In [118]:
toxic_list_all = {k: v for k, v in sorted(freqTable3.items(),reverse=True, key=lambda item: item[1])}
toxic_list_all

{'ass': 976,
 'fuck': 888,
 'suck': 370,
 'mexicans': 356,
 'fucking': 272,
 'gay': 229,
 'nigger': 187,
 'die': 162,
 'must': 162,
 'jim': 157,
 'wales': 156,
 'cuntbag': 126,
 '====': 125,
 'shit': 115,
 'bastard': 114,
 'pro-assad.hanibal911you': 106,
 'eat': 99,
 'admins': 99,
 'hate': 99,
 'cocksucking': 96,
 'cunts': 94,
 'jews': 92,
 'niggers': 81,
 'spics': 79,
 'minoritiesi': 79,
 'abusing': 78,
 'latinus': 76,
 'powerproof': 74,
 'like': 57,
 'go': 47,
 'people': 46,
 'get': 38,
 'fucker': 38,
 'talk': 37,
 'penis': 37,
 'know': 35,
 'mother': 35,
 'ban': 35,
 'freak': 32,
 'hey': 32,
 'think': 32,
 'curps': 32,
 'one': 31,
 'shoot': 31,
 'unban': 29,
 'would': 26,
 'na': 24,
 'hell': 21,
 'life': 21,
 'bitch': 21,
 'yeah': 21,
 'ca': 20,
 'time': 20,
 '.': 20,
 'stop': 19,
 'never': 17,
 'even': 17,
 'feel': 17,
 'block': 16,
 'see': 16,
 'going': 16,
 'good': 16,
 'first': 16,
 'want': 16,
 'back': 16,
 'wiki': 15,
 'stupid': 15,
 'really': 15,
 'asshole': 15,
 'dick': 15,


In [119]:
toxic_top15 = ['ass','fuck','suck','mexicans','fucking','gay','nigger','die','must','jim','wales','cuntbag','===','shit','bastard']

In [120]:
toxic_top15

['ass',
 'fuck',
 'suck',
 'mexicans',
 'fucking',
 'gay',
 'nigger',
 'die',
 'must',
 'jim',
 'wales',
 'cuntbag',
 '===',
 'shit',
 'bastard']

In [121]:
X_test5_all_2_filtered = X_test5_all_2[X_test5_all_2['toxic_pred'] == 1]

In [128]:
X_test5_all_2_filtered = X_test5_all_2_filtered.drop(['toxic_pred'], axis=1)

In [129]:
toxic_df = X_test5_all_2_filtered.transpose()
toxic_df

,18,26,32,38,40,56,61,68,71,72,...,4949,4951,4962,4970,4971,4980,4983,4985,4989,4991
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
јули,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
јун,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
јуни,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
கள,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
toxic_df['sum'] = toxic_df.sum(axis=1)
toxic_df

,18,26,32,38,40,56,61,68,71,72,...,4951,4962,4970,4971,4980,4983,4985,4989,4991,sum
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.606199
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
0000z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.374334
02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
јули,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
јун,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
јуни,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
கள,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [131]:
toxic_df_top15 = toxic_df.nlargest(15,'sum')
toxic_df_top15

,18,26,32,38,40,56,61,68,71,72,...,4951,4962,4970,4971,4980,4983,4985,4989,4991,sum
fuck,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.220921,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.177293,0.0,25.506751
fucking,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.598752,0.000000,0.0,17.282863
go,0.000000,0.0,0.0,0.0,0.444011,0.0,0.0,0.000000,0.0,0.0,...,0.167931,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,10.721418
gay,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,9.074074
like,0.150836,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.248865,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,9.042856
ass,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,8.865383
hey,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.822947,0.0,0.0,...,0.218347,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.175228,0.0,8.210776
get,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.142400,0.000000,0.0,7.928830
shit,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,7.857516
people,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,7.669021


In [132]:
toxic_df_top15.index

Index(['fuck', 'fucking', 'go', 'gay', 'like', 'ass', 'hey', 'get', 'shit',
       'people', 'suck', 'asshole', 'think', 'hell', 'dick'],
      dtype='object')